# Measure time

### Latence Video to frames

In [1]:
import time
from LatenceMeasure import videoToFramesTIME_1, videoToFramesTIME_2, videoToFramesTIME_3, videoToFramesTIME_4
game = '/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv'

In [2]:
game1 ='/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/2_224p.mkv'
game2 ='/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv'
game3 ='/scratch/users/mdelabrassinne/Database/germany_bundesliga/2015-2016/2015-08-22_-_16-30_Hoffenheim_1_-_2_Bayern_Munich/1_224p.mkv'
game4 ='/scratch/users/mdelabrassinne/Database/germany_bundesliga/2015-2016/2015-08-22_-_16-30_Hoffenheim_1_-_2_Bayern_Munich/2_224p.mkv'
game5 ='/scratch/users/mdelabrassinne/Database/germany_bundesliga/2015-2016/2015-09-20_-_18-30_Dortmund_3_-_0_Bayer_Leverkusen/2_224p.mkv'

In [2]:
# Test 1
time = videoToFramesTIME_1(game)
print(time)

Folder Test_Frames removed.
0:07:47.156199


In [3]:
# Test on multiple videos
time = videoToFramesTIME_1(game1)
print(time)
time = videoToFramesTIME_1(game2)
print(time)
time = videoToFramesTIME_1(game3)
print(time)
time = videoToFramesTIME_1(game4)
print(time)
time = videoToFramesTIME_1(game5)
print(time)

Folder Test_Frames removed.
0:08:08.130359
Folder Test_Frames removed.
0:08:01.833090
Folder Test_Frames removed.
0:07:46.231922
Folder Test_Frames removed.
0:08:07.273747
Folder Test_Frames removed.
0:08:43.983727


In [12]:
t = (8+8*60+8*60+1+7*60+46+8*60+7+8*60+43)/5
print(t)
print(8,"min", t-8*60, "sec")

489.0
8 min 9.0 sec


In [5]:
# Test 2
time = videoToFramesTIME_2(game)
print(time)

Frames extraction ended.
Folder Test_Frames removed.
0:08:54.839921


In [3]:
time = videoToFramesTIME_2(game1)
print(time)
time = videoToFramesTIME_2(game2)
print(time)
time = videoToFramesTIME_2(game3)
print(time)
time = videoToFramesTIME_2(game4)
print(time)
time = videoToFramesTIME_2(game5)
print(time)

Frames extraction ended.
Folder Test_Frames removed.
0:08:35.139115
Frames extraction ended.
Folder Test_Frames removed.
0:08:14.116510
Frames extraction ended.
Folder Test_Frames removed.
0:08:28.707916
Frames extraction ended.
Folder Test_Frames removed.
0:07:35.653772
Frames extraction ended.
Folder Test_Frames removed.
0:11:22.685895


In [13]:
t = (8*60*3+7*60+11*60+35+14+28+35+22)/5
print(t)
print(8,"min", t-8*60, "sec")

530.8
8 min 50.799999999999955 sec


The first test uses the cv2 library while le second one uses the moviepy.editor. Both have been tested with the same video, a game of 45 min in real conditions. Comparing the two methods, we conclude that the first one seems faxter than the second.

In [2]:
# Test 3
time = videoToFramesTIME_3(game)
print(time)

Frames extraction ended.
Folder Test_Frames removed.
0:01:15.803788


A thirs library have been used to extract the frames bu this one is much more slower. For the first 500 frames, 1 min and 15 seconds are necessary while it was was around 6-7 seconds for the two other above.

The library that we will consider for this action of extraction is the open-cv one. It is said that for a 45 min game video, Less than 8 min are necessary to extract all the frames.

In [2]:
# sur 1000 frames
time = videoToFramesTIME_1(game)
print(time)

time = videoToFramesTIME_2(game)
print(time)

Folder Test_Frames removed.
0:00:05.369473
Frames extraction ended.
Folder Test_Frames removed.
0:00:05.605115


In [2]:
# Test 4 - sur 1500 frames
time = videoToFramesTIME_4(game)
print(time)

Frames extraction ended.
Folder Test_Frames removed.
0:00:44.379960


Une dernière méthode a été utilisée et elle n'est absolument pas concluante lorsqu'on compère les temps avec les deux premières

### Latence of the model

Measure the latence between the send of a sequence of a game and the reception of the answer by the model.

In [1]:
import numpy as np
from tqdm import tqdm
import os
import json
from Network import Network

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision
import math
import os
import copy
import time
import sqlite3 as sql
import cv2
from tqdm import tqdm
from FeaturesData import getPathFeatures, getFeatures, getFeaturesWindow
#from LatenceMeasure import displayMeasureTime, measureTimeWindows

In [2]:
# Select a footblall game in the database
db_path = "/scratch/users/mdelabrassinne/Database/SoccerDB.db"
#game = '/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv'
# A game in the test set

In [50]:
import copy
import json
import os
import numpy as np
import torch
import timeit
from timeit import default_timer as timer
from datetime import timedelta

class Results:

    def __init__(self, dir_model, features, window_size, classes, nms_window,fps=1):
        self.dir_model = dir_model
        self.features = features
        self.window_size = window_size
        self.nb_classes = len(classes)
        self.classes = classes
        self.nms_window = nms_window
        self.fps = fps

    def ResultsMeasureTime(self, disp_path, conf_path):
        dir_results = "/scratch/users/mdelabrassinne/Results_Test"
        
        conf_mod_path = os.path.join(self.dir_model, conf_path)
        disp_mod_path = os.path.join(self.dir_model, disp_path)
        model = Network(conf_mod_path, disp_mod_path)
        models = model.get_models()
        
        if not os.path.exists(dir_results):
            os.mkdir(dir_results)
            
        pred_files = []
        features = np.load(self.features)    
        
        count = 0
        for i in range(0, features.shape[0], self.window_size):
            if count > 5:
                break
            start = timer()
            conf1 = self.compute_confidences(features, i, models)

            # write the results in a json file
            dir_match = os.path.dirname("test")
            json_file_name = self.writeResults(conf1, dir_results, dir_match)
            pred_files.append(json_file_name)
            
            end = timer()
            print(f"Time: {timedelta(seconds=end-start)}")
            
            # Wait for the time to get futur features of match
            #ime.sleep(self.window_size)
            count += 1
        
    def compute_confidences(self, features, i, models):
        conf = np.zeros((features.shape[0], self.nb_classes))
        # Make the predictions
        if i+self.window_size >= features.shape[0]:
            seq = features[-self.window_size:]
            seq = np.expand_dims(seq, axis=0)
            seq = torch.Tensor(seq)
            seq = seq.transpose(1, 2)
            confidences = (
                (models[0](seq)).detach().numpy()).transpose()
            confidences = np.squeeze(confidences, axis=2)
            confidences = 1 / (1 + np.exp(-confidences))
            displacements = (
                (models[1](seq)).detach().numpy()).transpose()
            displacements = np.squeeze(displacements, axis=2)
            end = features.shape[0] - i
            conf[i:] = self.merge_nms(
                confidences[-end:], displacements[-end:])
        else:
            seq = features[i:i+self.window_size]
            seq = np.expand_dims(seq, axis=0)
            seq = torch.Tensor(seq)
            seq = seq.transpose(1, 2)
            confidences = (
                (models[0](seq)).detach().numpy()).transpose()
            confidences = np.squeeze(confidences, axis=2)
            confidences = 1 / (1 + np.exp(-confidences))
            displacements = (
                (models[1](seq)).detach().numpy()).transpose()
            displacements = np.squeeze(displacements, axis=2)
            conf[i:i +
                 self.window_size] = self.merge_nms(confidences, displacements)
            
        return conf

    
    def writeResults(self, conf: np.ndarray,  dir_results: str, dir_match: str):
        """
        Write the results in a JSON file.

        Args:
            param conf: confidence of the model
            param dir_results: directory of the results
        """

        def add_action(self, actions: list, confidences: np.ndarray, half: int) -> list:
            for i in range(len(confidences)):
                probas = confidences[i]
                time = i / self.fps
                min = int(time/60)
                sec = int((time) % 60)
                for c, proba in enumerate(probas):
                    if proba != 0.0:
                        gt = str(half) + " - " + str(min).zfill(2) + \
                            ":" + str(sec).zfill(2)
                        name_action = self.classes[c]
                        actions.append({})
                        actions[-1]["label"] = name_action
                        actions[-1]["position"] = time * \
                            1000  # time in millseconds
                        actions[-1]["half"] = half
                        actions[-1]["confidence"] = proba
                        actions[-1]["gameTime"] = gt

            return actions

        my_path = dir_results
        match_name = ""
        league_names = ["england_epl", "europe_uefa-champions-league",
                        "france_ligue-1", "germany_bundesliga", "italy_serie-a"]
        for league in league_names:
            if league in dir_match:
                match_name = league
                break

        season_year = ["2014-2015", "2015-2016", "2016-2017",
                       "2017-2018", "2018-2019", "2019-2020"]
        for season in season_year:
            if season in dir_match:
                my_path = os.path.join(my_path, season)
                match_name = os.path.join(match_name, season)
                break
        match_string = copy.deepcopy(dir_match[(len(my_path)+1):-11])
        match_string = match_string.replace("_", " ")

        my_path = os.path.join(my_path, match_string)
        match_name = os.path.join(match_name, match_string)

        json_dict = {}

        # Define the half based on the name of the video that is being processed
        half = 1 if "1_224p.mkv" or "1_720p.mkv" in dir_match else 2

        json_dict["UrlLocal"] = match_name
        json_dict["predictions"] = add_action(self, [], conf, half)

        json_object = json.dumps(json_dict, indent=4)

        json_file_name = os.path.join(dir_match, f"{match_name}.json")
        # match name equals to path of last folder
        match_name = match_name.split("/")[-1]

        # Writing
        json_file_name = os.path.join(dir_results, f"{match_name}.json")

        # Writing
        with open(json_file_name, "w") as outfile:
            outfile.write(json_object)

        return json_file_name
    
    def compute_update_confidence(self, confidences: np.ndarray, displacements: np.ndarray) -> np.ndarray:
        """
        Computes the updated confidences using the displacements.

        Args:
            confidences (np.ndarray): Confidence of each class for each time step of each video.
            displacements (np.ndarray): Displacement of each class for each time step of each video.

        Returns:
            np.ndarray: Updated confidences of each class for each time step of each video.
        """
        T = confidences.shape[0]
        K = confidences.shape[1]

        displacements = np.round(displacements * self.fps).astype(int)
        output = np.zeros_like(confidences)
        classes = np.arange(K)

        for t in range(T):
            disp = displacements[t, :]
            idx = t - disp
            idx = np.clip(idx, 0, T - 1)
            output[idx, classes] = np.maximum(output[idx, classes], confidences[t, classes])
        
        return output    
    
    def merge_nms(self, confidences: np.ndarray, displacements: np.ndarray) -> np.ndarray:
        """
        Evaluates the updated confidences using the displacements and Normalized Maximum Suppression (NMS) on time series.

        Args:
            confidences (np.ndarray): Confidence of each class for each time step of each video.
            displacements (np.ndarray): Displacement of each class for each time step of each video. The displacements 
                are in seconds.
        Returns:
            np.ndarray: Updated confidences of each class for each time step of each video.
        """

        # Check the shape of the confidences and displacements
        assert confidences.shape == displacements.shape, "The shape of the confidences and displacements should be the same."

        # 1. Compute the updated confidences using the displacements
        # 2. Compute the NMS on the time series

        # 1. Compute the updated confidences using the displacements
        updated_confidences = self.compute_update_confidence(confidences, displacements)

        # 2. Compute the NMS on the time series
        nms_confidences = self.compute_nms(updated_confidences)
        return nms_confidences
       

    def compute_nms(self, confidences: np.ndarray) -> np.ndarray:
        """
        Computes the NMS on the time series for each class seperately.

        Args:
            confidences (np.ndarray): Updated confidences of each class for each time step of each video.

        Returns:
            np.ndarray: NMS confidences of each class for each time step of each video.
        """
        T = confidences.shape[0]
        K = confidences.shape[1]

        output = confidences.copy()

        for k in range(K):
            nms_window = self.nms_window[k]
            nms_window = int(nms_window * self.fps)
            preds = confidences[:, k]
            copy_preds = preds.copy()

            # Apply NMS for a window of size nms_window seconds
            max_idx = np.argmax(copy_preds)
            max_val = copy_preds[max_idx]

            while max_val > 0:
                start = max(0, max_idx - nms_window)
                end = min(T, max_idx + nms_window + 1)

                preds[start:end] = 0
                preds[max_idx] = max_val

                copy_preds[start:end] = 0
                max_idx = np.argmax(copy_preds)
                max_val = copy_preds[max_idx]
            
            output[:, k] = preds
        
        return output

In [51]:
dir_model = "/home/mdelabrassinne/DSproject/Milestone3/Time Measures"
classes = ['Kick-off', 
           'Ball out of play', 
           'Throw-in', 'Foul', 
           'Clearance', 
           'Indirect free-kick', 
           'Shots on target', 
           'Corner', 
           'Shots off target', 
           'Direct free-kick', 
           'Yellow card', 
           'Substitution', 
           'Red card', 
           'Offside', 
           'Goal', 
           'Penalty', 
           'Yellow->red card']

game1 = '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2016-01-03_-_16-30_Crystal_Palace_0_-_3_Chelsea/1_224p.mkv'
game2 = '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2016-01-03_-_16-30_Crystal_Palace_0_-_3_Chelsea/2_224p.mkv'
game3 = '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2016-03-02_-_23-00_Liverpool_3_-_0_Manchester_City/1_224p.mkv'
game4 = '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2016-03-02_-_23-00_Liverpool_3_-_0_Manchester_City/2_224p.mkv'
game5 = '/scratch/users/mdelabrassinne/Database/italy_serie-a/2016-2017/2017-04-30_-_21-45_Inter_0_-_1_Napoli/2_720p.mkv'

games = [game1, game2, game3, game4, game5]

nms_windows = [20 for i in classes]

In [52]:
from timeit import default_timer as timer
from datetime import timedelta

#Measure the time necessary for an entire half-time for 64 seconds window

conf = "confidence_window_64.pt"
disp = "disp_window_64.pt"
window_size = 64

for game in games:
    print(games.index(game))
    features = getPathFeatures(game)
    res = Results(dir_model, features, window_size, classes, nms_windows)
    res.ResultsMeasureTime(disp,conf)
    

0
Time: 0:00:00.087865
Time: 0:00:00.087808
Time: 0:00:00.088771
Time: 0:00:00.088727
Time: 0:00:00.087732
Time: 0:00:00.087578
1
Time: 0:00:00.088967
Time: 0:00:00.088592
Time: 0:00:00.088398
Time: 0:00:00.090602
Time: 0:00:00.089313
Time: 0:00:00.087397
2
Time: 0:00:00.087509
Time: 0:00:00.087993
Time: 0:00:00.087289
Time: 0:00:00.089631
Time: 0:00:00.087535
Time: 0:00:00.088601
3
Time: 0:00:00.122962
Time: 0:00:00.101883
Time: 0:00:00.118056
Time: 0:00:00.120906
Time: 0:00:00.125097
Time: 0:00:00.103672
4
Time: 0:00:00.124112
Time: 0:00:00.108663
Time: 0:00:00.112843
Time: 0:00:00.113727
Time: 0:00:00.087336
Time: 0:00:00.089417


In [56]:
((0.087 * 4 + 0.088 * 2)/6 + (0.088 * 3 + 0.09 + 0.089 + 0.087)/6 + (0.088+ 0.089 + 0.087*4)/6 + (0.12*3+0.11+0.10*2)/6 + (0.12+0.10+0.11*2+0.087+0.089)/6) / 5

0.09549999999999999

In [53]:
conf = "confidence_window_112.pt"
disp = "disp_window_112.pt"
window_size = 112

for game in games:
    print(games.index(game))
    features = getPathFeatures(game)
    res = Results(dir_model, features, window_size, classes, nms_windows)
    res.ResultsMeasureTime(disp,conf)

0
Time: 0:00:00.141851
Time: 0:00:00.142116
Time: 0:00:00.127915
Time: 0:00:00.139105
Time: 0:00:00.144810
Time: 0:00:00.141799
1
Time: 0:00:00.127874
Time: 0:00:00.123921
Time: 0:00:00.147409
Time: 0:00:00.116441
Time: 0:00:00.111870
Time: 0:00:00.110445
2
Time: 0:00:00.118502
Time: 0:00:00.112254
Time: 0:00:00.114726
Time: 0:00:00.112082
Time: 0:00:00.113401
Time: 0:00:00.111067
3
Time: 0:00:00.109731
Time: 0:00:00.109343
Time: 0:00:00.113543
Time: 0:00:00.109401
Time: 0:00:00.111699
Time: 0:00:00.110644
4
Time: 0:00:00.109474
Time: 0:00:00.111711
Time: 0:00:00.111846
Time: 0:00:00.109461
Time: 0:00:00.112186
Time: 0:00:00.109957


In [57]:
( (0.14*4+0.12+0.13)/6 + (0.12*2+0.11*3+0.14)/6 + (0.118+0.112+0.114+0.112+0.113+0.111)/6 + (0.109+0.109+0.113+0.1109+0.111+0.1106)/6 + (0.109+0.1117+0.1118+0.1094+0.112+0.1099)/6 )/5

0.11757666666666669

In [54]:
conf = "confidence_window_32.pt"
disp = "disp_window_32.pt"
window_size = 32

for game in games:
    print(games.index(game))
    features = getPathFeatures(game)
    res = Results(dir_model, features, window_size, classes, nms_windows)
    res.ResultsMeasureTime(disp,conf)

0
Time: 0:00:00.077210
Time: 0:00:00.076466
Time: 0:00:00.075749
Time: 0:00:00.077634
Time: 0:00:00.075485
Time: 0:00:00.076246
1
Time: 0:00:00.073899
Time: 0:00:00.075747
Time: 0:00:00.076204
Time: 0:00:00.076518
Time: 0:00:00.075433
Time: 0:00:00.076104
2
Time: 0:00:00.074964
Time: 0:00:00.075967
Time: 0:00:00.076354
Time: 0:00:00.078829
Time: 0:00:00.076930
Time: 0:00:00.076476
3
Time: 0:00:00.075116
Time: 0:00:00.075943
Time: 0:00:00.075692
Time: 0:00:00.078085
Time: 0:00:00.074653
Time: 0:00:00.077221
4
Time: 0:00:00.075053
Time: 0:00:00.076739
Time: 0:00:00.076798
Time: 0:00:00.077583
Time: 0:00:00.076041
Time: 0:00:00.077463


In [58]:
((0.077*2+0.076*2+0.075*2)/6+(0.076*3+0.075*2+0.073)/6+(0.0749+0.0759+0.0769+0.0788+0.0769+0.0764)/6+(0.07511+0.07594+0.07569+0.07808+0.07465+0.07722)/6+(0.07505+0.07673+0.076798+0.07758+0.07604+0.07746)/6)/5

0.07610493333333333